In [1]:
from myosuite.utils import gym
import skvideo.io
import numpy as np
import os

MyoSuite:> Registering Myo Envs


In [2]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):

  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")


In [3]:
pth = '../../../myosuite/agents/baslines_NPG/'

policy = pth+"myoElbowPose1D6MExoRandom-v0/2022-02-26_21-16-27/36_env=myoElbowPose1D6MExoRandom-v0,seed=1/iterations/best_policy.pickle"

import pickle
pi = pickle.load(open(policy, 'rb'))


In [4]:
env = gym.make('myoElbowPose1D6MExoRandom-v0')

env.reset()


libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: failed to open /dev/dri/renderD128: Permission denied

libEGL warning: NEEDS EXTENSION: falling back to kms_swrast


    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


(array([0.801 , 0.    , 0.2991, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    ], dtype=float32),
 {})

In [5]:
# define a discrete sequence of positions to test
AngleSequence = [60, 30, 30, 60, 80, 80, 60, 30, 80, 30, 80, 60]
env.reset()
frames = []
for ep in range(len(AngleSequence)):
    print("Ep {} of {} testing angle {}".format(ep, len(AngleSequence), AngleSequence[ep]))
    env.unwrapped.target_jnt_value = [np.deg2rad(AngleSequence[int(ep)])]
    env.unwrapped.target_type = 'fixed'
    env.unwrapped.weight_range=(0,0)
    env.unwrapped.update_target()
    for _ in range(40):
        frame = env.sim.renderer.render_offscreen(
                        width=400,
                        height=400,
                        camera_id=0)
        frames.append(frame)
        o = env.get_obs()
        a = pi.get_action(o)[0]
        next_o, r, done, *_, ifo = env.step(a) # take an action based on the current observation
env.close()

os.makedirs('videos', exist_ok=True)
# make a local copy
skvideo.io.vwrite('videos/exo_arm.mp4', np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})


Ep 0 of 12 testing angle 60


/home/xliu227/miniconda3/envs/myosuite/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
/home/xliu227/miniconda3/envs/myosuite/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_obs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_obs` for environment variables or `env.get_wrapper_attr('get_obs')` that will search the reminding wrappers.
  logger.warn(


Ep 1 of 12 testing angle 30
Ep 2 of 12 testing angle 30
Ep 3 of 12 testing angle 60
Ep 4 of 12 testing angle 80
Ep 5 of 12 testing angle 80
Ep 6 of 12 testing angle 60
Ep 7 of 12 testing angle 30
Ep 8 of 12 testing angle 80
Ep 9 of 12 testing angle 30
Ep 10 of 12 testing angle 80
Ep 11 of 12 testing angle 60


AssertionError: Cannot find installation of real FFmpeg (which comes with ffprobe).

In [ ]:
show_video('videos/exo_arm.mp4')